In [1]:
import timm
from fastai.vision.all import *
from fastcore.parallel import *

In [2]:
path = Path.BASE_PATH = Path.home()/'paddy'
trn_path = path/'train_images'
tst_files = get_image_files(path/'test_images').sorted()

In [3]:
tta_res = []

In [4]:
def train(desc, arch, item, batch, accum=False):
    kwargs = {'bs': 32} if accum else {}
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item, batch_tfms=batch, **kwargs)
    cbs = GradientAccumulation(2) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    learn.fine_tune(12, 0.01)
    tts_res.append(learn.tta(learn.tta(dl=dls.test_dl(tst_files))))
    learn.export(f'{arch}_{desc}')